In [1]:
import requests
from lxml import html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
from time import sleep

#Link de información conjunta sobre nominación a premios
nominaciones = "https://es.wikipedia.org/wiki/Rick_y_Morty"
#Encabezado posiblemente requeridos en proceso de scraping
encabezado = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

#Acceder a resquest
noms = requests.get(nominaciones, headers= encabezado)

#Parsear el árbol html
soup = bs(noms.text, "html.parser")
#Buscar el elemento que se desea extraer, en este caso una tabla
tabla = soup.find("table", {"class" : "wikitable sortable"})
#Convertir directamente la tabla a dataframe a través de pandas y read_html
tabla = pd.read_html(str(tabla))[0]
#En este caso la última columna es de referencias de nominaciones, no son útiles en este formato deseado
eliminar = "Ref."
tablanominaciones = tabla.drop(eliminar, axis = 1)

#Acá el resultado incluye unas celdas combinadas con la temporada correspondiente de la nominaciones
#No obstate se hace necesario convertir estos en una nueva columna y tratar de eliminar estas filas
 
temporada_actual = None #Se inicializa una lista vacía para guardar los valores de lectura

temporadas = [] #Una segunda lista vacía para almacenamientos de valores

#Se navega cuidadosamente a través de las filas, con ayuda de la primera columna dado que en todas son iguales
for index, fila in tablanominaciones.iterrows():
    if fila["Año"].startswith("Temporada"):
        temporada_actual = fila["Año"]
    else:
        temporada = temporada_actual
        fila["Temporada"] = temporada_actual
        temporadas.append(fila)

tablanominaciones = pd.DataFrame(temporadas)
tablanominaciones


,Año,Premiación,Categoría,Nominado(s),Resultado,Temporada
1,2014,BTVA Voice Acting Awards,Best Male Vocal Performance in a Television Se...,Chris Parnell,Nom,Temporada 1
2,2014,BTVA Voice Acting Awards,Best Female Vocal Performance in a Television ...,Sarah Chalke,Nom,Temporada 1
3,2014,IGN Awards,Best TV Animated Series,Rick and Morty,Nom,Temporada 1
4,2014,IGN People's Choice Award,Best TV Animated Series,Rick and Morty,Nom,Temporada 1
5,2015,BTVA Voice Acting Awards,Best Male Lead Vocal Performance in a Televisi...,Justin Roiland,Ganador,Temporada 1
6,2015,Annie Awards,Best General Audience Animated TV/Broadcast Pr...,Rick and Morty,Nom,Temporada 1
8,2015,IGN Awards,Best Animated Series,Rick and Morty,Ganador,Temporada 2
9,2015,IGN People's Choice Award,Best Animated Series,Rick and Morty,Ganador,Temporada 2
10,2016,BTVA Voice Acting Awards,Best Male Lead Vocal Performance in a Televisi...,Justin Roiland,Nom,Temporada 2
11,2016,BTVA Voice Acting Awards,Best Vocal Ensemble in a Television Series,Rick and Morty,Nom,Temporada 2


In [2]:
#Link de nominaciones más completa de la serie
# Configura el controlador de Selenium (asegúrate de tenerlo instalado)
options = Options()
encabezado = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
options.add_argument(f"user-agent={encabezado}")
driver = webdriver.Chrome(options=options)  # Debes tener el controlador de Chrome descargado y en tu PATH
#driver = webdriver.Chrome()
# Abre la página web
rickymortynoms = "https://www.imdb.com/title/tt2861424/awards"  # Reemplaza con la URL real

driver.get(rickymortynoms)

# Encuentra el botón "Ver más" por su clase
ver_mas_button = driver.find_elements(By.XPATH, "//button[@class= 'ipc-btn ipc-btn--single-padding ipc-btn--center-align-content ipc-btn--default-height ipc-btn--core-base ipc-btn--theme-base ipc-btn--on-accent2 ipc-text-button ipc-see-more__button']")

for item in ver_mas_button:
    if "más" in item.text:
        #print(item.text)
        driver.execute_script("arguments[0].click();", item)
#Obtener html
sleep(random.uniform(8.0, 12.0))
nominaciones1 = driver.page_source
driver.quit()

#nominaciones1 = requests.get(rickymortynoms, headers= encabezado)

soup1 = bs(nominaciones1, "html.parser")

añonomninacion = soup1.find_all(class_ = "ipc-metadata-list-summary-item__t")
categoriapremiacion = soup1.find_all(class_ = "ipc-metadata-list-summary-item__li awardCategoryName")
episodio = soup1.find_all(class_ = "ipc-md-link ipc-md-link--entity")

nomins = []
for nomi, cat in zip(añonomninacion, categoriapremiacion):
    año = nomi.text.split()[0]
    resultado = nomi.text.split()[1]
    premiacion = ' '.join(nomi.text.split()[2:])
    categoria = cat.text
    nomins.append((año, resultado, premiacion, categoria))

nomins = pd.DataFrame(nomins, columns=["Año", "Resultado", "Premiación", "Categoría"])
nomins


,Año,Resultado,Premiación,Categoría
0,2021,Nominado,Saturn Award,Best Animated Series on Television
1,2018,Nominado,Saturn Award,Best Animated Series or Film on Television
2,2022,Nominado,Eddie,Best Edited Animation - Non-Theatrical
3,2021,Ganador,Eddie,Best Edited Animation - Non-Theatrical
4,2023,Nominado,Annie,Best General Audience Animated Television/Medi...
5,2021,Nominado,Annie,Best General Audience Animated Television/Medi...
6,2018,Ganador,Annie,Outstanding Achievement for Writing in an Anim...
7,2018,Ganador,Annie,Best General Audience Animated Television/Broa...
8,2015,Nominado,Annie,Best General Audience Animated TV/Broadcast Pr...
9,2018,Ganador,Critics Choice Award,Best Animation Series


In [85]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import random
from time import sleep
# Configura el controlador de Selenium (asegúrate de tenerlo instalado)
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)  # Debes tener el controlador de Chrome descargado y en tu PATH

# Abre la página web
url = "https://www.imdb.com/title/tt2861424/awards"  # Reemplaza con la URL real
driver.get(url)

# Encuentra el botón "Ver más" por su clase
ver_mas_button = driver.find_elements(By.XPATH, "//button[@class= 'ipc-btn ipc-btn--single-padding ipc-btn--center-align-content ipc-btn--default-height ipc-btn--core-base ipc-btn--theme-base ipc-btn--on-accent2 ipc-text-button ipc-see-more__button']")


for item in ver_mas_button:
    if "más" in item.text:
        print(item.text)
        #item.click()
        scroll_to_element(item)
        driver.execute_script("arguments[0].click();", item)

        # Haz clic en el elemento usando una acción de teclado
    #keyboard_action.perform()
        sleep(random.uniform(8.0, 11.0))
    #else:
    #    continue

#Obtener html
nominaciones1 = driver.page_source
driver.quit()
#print(nominaciones1)


7 más
2 más
